In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
font = {'size': 16}
matplotlib.rc('font', **font)
tqdm.tqdm.pandas()

In [ ]:
%%time
main_data_df = pd.read_csv('./Prepped Data/main_data_range_implement_20220103.csv')
print(main_data_df.shape)
main_data_df.head()

In [ ]:
main_data_df['구'] = main_data_df['시군구'].apply(lambda x: x.split(' ')[1])
main_data_df['동'] = main_data_df['시군구'].apply(lambda x: x.split(' ')[2])

In [ ]:
main_data_df['층'] = main_data_df['층'].astype('Int64')

In [ ]:
main_data_df['건축년도'] = main_data_df['건축년도'].astype('Int64')

In [ ]:
main_data_df['계약날짜일수_2006년기준'] = main_data_df['계약날짜일수_2006년기준'].astype('Int64')

In [ ]:
main_data_df.head()

In [ ]:
main_data_df.isna().sum()

In [ ]:
main_data_df['주소_건축년도'] = main_data_df['시군구'] + ' ' + main_data_df['지번'].astype('str') + ' ' + main_data_df['건축년도'].astype('str')

In [ ]:
main_data_df['토지면적'].apply(np.log).hist(bins=200)

In [ ]:
main_data_df['토지면적'].apply(np.log1p).hist(bins=200)

In [ ]:
main_data_df['토지면적_log'] = main_data_df['토지면적'].apply(np.log)

In [ ]:
main_data_df['건축년도'].value_counts()

In [ ]:
main_data_df['건축년도'] = main_data_df['건축년도'].apply(lambda x: '1980이하' if x <= 1980 else str(x))

In [ ]:
main_data_df['건축년도'].value_counts()

In [ ]:
main_data_df['층'].value_counts()

In [ ]:
def floor_feature_engi(x):
    # x is a number
    
    if x < 0:
        return '지하'
    elif x >= 30:
        return '30층이상'
    else:
        return str(x)

In [ ]:
main_data_df['층'] = main_data_df['층'].apply(floor_feature_engi)

In [ ]:
main_data_df['층'].value_counts()

In [ ]:
main_data_df.head()

In [ ]:
cat_cols = ['거래유형', '층', '건축년도', '부동산유형', '브랜드', '대장구분명', '지목명', '용도지역명1', '용도지역명2',
            '토지이동상황', '지형높이', '지형형상', '도로접면', '구', '동']

In [ ]:
for col in tqdm.tqdm_notebook(cat_cols):
    unique_cats = main_data_df[col].unique()
    
    # mapping
    cat_map = {}
    for i, v in enumerate(unique_cats):
        cat_map[v] = i
        
    main_data_df[col] = main_data_df[col].map(cat_map)
    
    map_df = pd.DataFrame(cat_map, index=[0])
    map_df.to_excel('./Training/Training2_20211230/cat_col_maps/{}'.format(col+'.xlsx'), index=False)

In [ ]:
training_data_df = main_data_df[['주소_건축년도', '전용면적(㎡)', '월세(만원)', 'x좌표', 'y좌표', '계약날짜', '계약날짜일수_2006년기준', '토지면적_log'] + cat_cols + ['거래/보증금(만원)']]
print(training_data_df.shape)
training_data_df.head()

In [ ]:
training_data_df['x좌표'] = training_data_df['x좌표'] / 100000
training_data_df['y좌표'] = training_data_df['y좌표'] / 100000

In [ ]:
training_data_df.head()

In [ ]:
%%time
training_data_df.to_csv('./Prepped Data/training_data_20211230.csv', index=False)